In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

In [2]:
result = pd.read_csv('result.csv', index_col=0)
da = pd.read_csv('DaycaresAnnotations_original.csv')
result['detection'] = 'daycare'
da['detection'] = 'daycare'

In [3]:
result.head(5)

,file,conf,x,y,xmin,ymin,xmax,ymax,width,height,detection
0,11926_20161008.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
1,11926_20170331.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
2,11946_20140215.jpg,0.597667,747.607422,422.410187,670.337044,339.748055,824.8778,505.072319,154.540756,165.324265,daycare
3,12645_20160829.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
4,12650_20120928.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare


In [4]:
ground_truth = []
for r in range(len(da)):
    if da.loc[r, 'region_shape_attributes'] != "{}":
        filename = da.loc[r, 'filename']
        rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])
        # x-min
        xmin = int(rect['x'])
        # x-max
        xmax = int(rect['x'] + rect['width'])
        # y-min
        ymin = int(rect['y'])
        # y-max
        ymax = int(rect['y'] + rect['height'])
        width = rect['width']
        height = rect['height']
        detection = da.loc[r, 'detection']
        ground_truth.append([filename, detection, xmin, ymax, xmax, ymin, width, height])
        
ground_truth = pd.DataFrame(np.array(ground_truth), columns = ['file', 'detection', 'xmin', 'ymax', 'xmax', 'ymin', 'width', 'height'])
detection_result = result[['file', 'detection', 'conf', 'x', 'y']]
positive_detection_result = detection_result
# positive_detection_result = detection_result.loc[~((detection_result['x'] == 0) & (detection_result['y'] == 0))]
# detection_result = result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']]
# positive_detection_result = detection_result.loc[~((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]
# zero_detection_result = detection_result.loc[((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]


In [5]:
positive_detection_result.head(5)

,file,detection,conf,x,y
0,11926_20161008.jpg,daycare,0.000000,0.000000,0.000000
1,11926_20170331.jpg,daycare,0.000000,0.000000,0.000000
2,11946_20140215.jpg,daycare,0.597667,747.607422,422.410187
3,12645_20160829.jpg,daycare,0.000000,0.000000,0.000000
4,12650_20120928.jpg,daycare,0.000000,0.000000,0.000000


In [6]:
# ground_truth[['file', 'width', 'height']]
positive_detection_result = pd.merge(ground_truth[['file', 'width', 'height']], positive_detection_result, on='file', how='inner')


In [7]:
positive_detection_result[['x', 'y']]

,x,y
0,612.489319,661.928040
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,885.267517,490.166351
5,0.000000,0.000000
6,708.641296,373.316650
7,701.306335,371.512543
8,700.160339,371.040649
9,701.965454,437.396851


In [12]:
# def calculate_area(row):
#     rou(int(row['x']) - int(row['width']) /2 )
#     return row['x'] - row['width'] / 2
positive_detection_result[['x', 'y']] = positive_detection_result[['x', 'y']].astype(int)
positive_detection_result['xmin'] = positive_detection_result.apply(
    lambda row: (int(row['x']) - round(int(row['width']) / 2)) if int(row['x']) > 0 else 0, axis=1)
positive_detection_result['xmax'] = positive_detection_result.apply(
    lambda row: (int(row['x']) + round(int(row['width']) / 2)) if int(row['x']) > 0 else 0, axis=1)
positive_detection_result['ymin'] = positive_detection_result.apply(
    lambda row: (int(row['y']) - round(int(row['height']) / 2)) if int(row['x']) > 0 else 0, axis=1)
positive_detection_result['ymax'] = positive_detection_result.apply(
    lambda row: (int(row['y']) + round(int(row['height']) / 2)) if int(row['x']) > 0 else 0, axis=1)


# positive_detection_result = positive_detection_result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']].drop_duplicates()
positive_detection_result = positive_detection_result[['file', 'detection', 'conf', 'ymin', 'xmin', 'ymax', 'xmax']].drop_duplicates()


In [13]:
positive_detection_result.head(5)

,file,detection,conf,ymin,xmin,ymax,xmax
0,17364_20140412.jpg,daycare,0.507108,621,562,701,662
1,17439_20160728.jpg,daycare,0.000000,0,0,0,0
2,17471_20160211.jpg,daycare,0.000000,0,0,0,0
3,17471_20160816.jpg,daycare,0.000000,0,0,0,0
4,17471_20170623.jpg,daycare,0.576698,448,845,532,925


In [14]:
positive_detection_files = positive_detection_result['file'].values
# zero_detection_result_files = zero_detection_result['file'].values
relevant_ground_truth = ground_truth[ground_truth['file'].isin(positive_detection_files)].drop_duplicates()


In [15]:
print(len(relevant_ground_truth), len(positive_detection_result))
print(set(relevant_ground_truth['file'].values) - set(positive_detection_result['file'].values))  
assert len(relevant_ground_truth) == len(positive_detection_result)

488 488
set()


In [16]:
positive_detection_result[['xmin', 'ymax', 'xmax', 'ymin']] = positive_detection_result[['xmin', 'ymax', 'xmax', 'ymin']].astype(int)
positive_detection_result.sort_values(by=['file']).head(5)

,file,detection,conf,ymin,xmin,ymax,xmax
295,11926_20161008.jpg,daycare,0.000000,0,0,0,0
296,11926_20170331.jpg,daycare,0.000000,0,0,0,0
297,11946_20140215.jpg,daycare,0.597667,379,693,465,801
298,12645_20160829.jpg,daycare,0.000000,0,0,0,0
299,12650_20120928.jpg,daycare,0.000000,0,0,0,0


In [17]:
relevant_ground_truth = relevant_ground_truth[['file','detection','ymin', 'xmin','ymax','xmax']]
relevant_ground_truth.sort_values(by=['file']).head(5)

,file,detection,ymin,xmin,ymax,xmax
1540,11926_20161008.jpg,daycare,79,444,171,549
1541,11926_20170331.jpg,daycare,79,444,171,549
1542,11946_20140215.jpg,daycare,377,697,463,806
1552,12645_20160829.jpg,daycare,364,662,450,779
1553,12650_20120928.jpg,daycare,284,715,365,810


In [19]:
def put_files(df, loc):
    values = [[value[0], np.array2string(value[1:], separator=' ')
       .replace('[', '').replace(']', '').replace("'", "").replace("\n", "")
     ]for value in df.values]
    for value in values:
        filepath_label = loc + value[0]
        filepath_label = filepath_label.replace('jpg', 'txt')
#         print(value[1])
        with open(filepath_label, 'w') as f:
            f.write("%s\n" % value[1])
put_files(positive_detection_result, './mAP_custom/input/detection-results/')
put_files(relevant_ground_truth, './mAP_custom/input/ground-truth/')